In [1]:
import os
import requests
import re
from pathlib import Path

To use the selenium and the ChromeDriver, you will have to:

- `conda install -c conda-forge selenium`

Then you need to download ChromeDriver, install it here: <br>
https://sites.google.com/a/chromium.org/chromedriver/downloads

Be sure you download the right Chrome version. <br>
If you don't know your chrome version, you just need to click at options, help, about chrome and then you will find the right version for you. 

Once you have the chromedriver downloaded, run thoses lines:

```
sudo mv chromedriver /usr/bin/chromedriver
sudo chown root:root /usr/bin/chromedriver
sudo chmod +x /usr/bin/chromedriver
```

In case you are using a Windows Environment, you need to add the chromedriver to `path`

In [3]:
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
driver = webdriver.Chrome(options=options)
driver.get("https://www.wikiaves.com.br/midias.php?tm=s&t=s&s=11644&o=mp")

In [4]:
# Testing if selenium is working properly 

title = driver.find_element_by_class_name('wa-registros-titulo')
complete_name = title.text.strip('Sons de ')
popular_name = complete_name.split(' ')[0]
cientific_name = complete_name[complete_name.find('(') + 1: complete_name.find(')')] 

print(complete_name)
print(popular_name)
print(cientific_name)

canário-da-terra (Sicalis flaveola)
canário-da-terra
Sicalis flaveola


In [5]:
# Testing the download

audios = driver.find_elements_by_tag_name('audio')
# Verify if the folder exists, if not, it will create it
Path("audios").mkdir(parents=True, exist_ok=True)

for n, audio_src in enumerate(audios, 0):
    url = audio_src.get_attribute('src')
    r = requests.get(url, allow_redirects=True)
    filename = 'audios/' + popular_name + "_" + str(n) + ".mp3"
    open(filename, 'wb').write(r.content)

# Step 1: 
Loading the page and preparing to download the birds from Lages's Region

In [6]:
driver.get("https://www.wikiaves.com.br/especies.php?t=c&c=4209300&o=5")
table = driver.find_element_by_tag_name('table')

In [7]:
table_body = table.find_element_by_tag_name('tbody')
table_rows = table_body.find_elements_by_tag_name('tr')

In [28]:
for num, element in enumerate(table_rows[:10], start=0):
    print(num, element.text)

0 
1 Threskiornithidae
Theristicus caudatus curicaca 0 77
2 
3 Thraupidae
Sicalis flaveola canário-da-terra 0 56
4 
5 Mimidae
Mimus saturninus sabiá-do-campo 0 48
6 
7 Falconidae
Milvago chimachima carrapateiro 0 33
8 
9 Strigidae
Athene cunicularia coruja-buraqueira 0 31


In [12]:
for row in table_rows:
    link_family = row.find_element_by_class_name('font-red')
    # link = row.find_element_by_tag_name('a')
    print(link_family.get_attribute('href'))

https://www.wikiaves.com.br/wiki/threskiornithidae
https://www.wikiaves.com.br/wiki/threskiornithidae
https://www.wikiaves.com.br/wiki/thraupidae
https://www.wikiaves.com.br/wiki/thraupidae
https://www.wikiaves.com.br/wiki/mimidae
https://www.wikiaves.com.br/wiki/mimidae
https://www.wikiaves.com.br/wiki/falconidae
https://www.wikiaves.com.br/wiki/falconidae
https://www.wikiaves.com.br/wiki/strigidae
https://www.wikiaves.com.br/wiki/strigidae
https://www.wikiaves.com.br/wiki/turdidae
https://www.wikiaves.com.br/wiki/turdidae
https://www.wikiaves.com.br/wiki/trochilidae
https://www.wikiaves.com.br/wiki/trochilidae
https://www.wikiaves.com.br/wiki/picidae
https://www.wikiaves.com.br/wiki/picidae
https://www.wikiaves.com.br/wiki/cracidae
https://www.wikiaves.com.br/wiki/cracidae
https://www.wikiaves.com.br/wiki/falconidae
https://www.wikiaves.com.br/wiki/falconidae
https://www.wikiaves.com.br/wiki/thraupidae
https://www.wikiaves.com.br/wiki/thraupidae
https://www.wikiaves.com.br/wiki/ralli

KeyboardInterrupt: 